# Creación de la base de datos de Personajes Mitológicos

Todos los datos para la creación de la base de datos han sido extraídos de [Wikipedia](https://es.wikipedia.org/).

1. Creación de la tabla con las mitologías disponibles y su región.
2. Obtención de la tabla con los personajes pertenecientes a categorías específicas.
3. Obtención de la tabla con los personajes pertenecientes a varias categorías.
4. Creación de:
    * Dataset de los personajes mitológicos con su nombre, mitología, región y texto de Wikipedia.
    * Matriz de adyacencia con las relaciones de los personajes mitológicos basada en los links de Wikipedia.

Se importan las librarías usadas.

In [1]:
import pandas as pd

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

import time
from itertools import compress
import re
import unicodedata

In [2]:
# Funcion para obtener el objeto *BeautifulSoup* de la url.

def get_url(url):
    req = requests.get(url)
    if req.status_code == 200:
        document = BeautifulSoup(req.text, "html.parser")
    else:
        document = None
    return document

## 1. Mitologías por región

Se crea un data frame con las mitologías que se scrapearán y la región a la que pertenecen.

### Obtención de las mitologías

El conjunto de mitologías se obtiene de diferentes formas:

1. [Hijos de la categoría "Mitología por cultura"](https://es.wikipedia.org/wiki/Especial:%C3%81rbolDeCategor%C3%ADas?target=Mitolog%C3%ADa_por_cultura&mode=all)
2. [Descendientes de la categoría "Mitología por continente"](https://es.wikipedia.org/wiki/Categoría:Mitología_por_continente)
3. [Anexo de deidades](https://es.wikipedia.org/wiki/Anexo:Deidades)
4. Mitologías añadidas directamente a la lista

#### Mitologías 1

Se usa *BeautifulSoup*.

In [4]:
url = "https://es.wikipedia.org/wiki/Especial:ÁrbolDeCategorías?target=Mitología_por_cultura&mode=all"

nodos = get_url(url).find("div", class_ = "CategoryTreeChildren").contents

mitologias1 = []
for nodo in nodos[3:]: # los primeros 3 elementos no contienen mitologias
    mitologias1.append(nodo.find("a").text)

In [5]:
mitologias1

['Mitología afroamericana',
 'Mitología amazónica',
 'Mitología de América del Norte',
 'Mitología árabe',
 'Mitología aborigen australiana',
 'Mitología bereber',
 'Mitología cananea',
 'Mitología cartaginesa',
 'Mitología celta',
 'Mitología cristiana',
 'Mitología egipcia',
 'Mitología eslava',
 'Mitología etrusca',
 'Mitología finlandesa',
 'Mitología germana',
 'Mitología griega',
 'Mitología guanche',
 'Mitología hinduista',
 'Mitología hopi',
 'Mitología igbo',
 'Mitología incaica',
 'Mitología inuit',
 'Mitología irania',
 'Mitología irlandesa',
 'Mitología itálica',
 'Mitología del judaísmo',
 'Mitología kushita',
 'Mitología malgache',
 'Mitología maorí',
 'Mitología mapuche',
 'Mitología masai',
 'Mitología mesoamericana',
 'Mitología mesopotámica',
 'Mitología íbera',
 'Mitología lituana',
 'Mitología maya',
 'Mitología mexica',
 'Mitología pigmea',
 'Mitología pueblo',
 'Mitología romana',
 'Mitología selknam',
 'Mitología taína',
 'Mitología túrquica',
 'Mitología venezol

#### Mitologías 2

Código válido a día 30/08/2021  
Se usa *selenium*.

In [6]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
There is no [win32] chromedriver for browser 93.0.4577 in cache
Get LATEST driver version for 93.0.4577
Trying to download new driver from https://chromedriver.storage.googleapis.com/93.0.4577.63/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Sergi\.wdm\drivers\chromedriver\win32\93.0.4577.63]


In [7]:
url = "https://es.wikipedia.org/wiki/Categoría:Mitología_por_continente"

driver.get(url)

In [9]:
# Funcion para comprobar si el nodo es una mitologia

def es_mitologia(mit_name):
    return(mit_name[:10] == "Mitología " and " en " not in mit_name)

In [10]:
# Funcion para comprobar si la mitologia ya está guardada en la lista

def en_lista(mit_name, lista):
    return(mit_name in lista)

In [11]:
# Funcion para comprobar si el nodo de una mitologia se puede desplegar

def expand_mitologia(mit_name):
    try:
        state = (driver
                 .find_element_by_xpath("//span[@data-ct-title='{}']"
                                        .format(mit_name
                                                .replace(" ", "_")))
                 .get_attribute("title"))
    except:
        return(False)
    return(state == "expandir")

**Tarda mucho en ejecutarse**  
Los nodos de la URL son desplegables, es decir, contienen subcategorías, por lo que en su interior contienen más mitologías. La estrategia es ir abriendo los desplegables de mitologías que no se han abierto anteriormente y así obtener todos los nombres de las mitologías. Cuando se abre un desplegable, hay que volver a recorrer todos los nodos debido al formato del HTML y cómo selenium encuentra los nodos.

In [12]:
mitologias2 = []

nodos = (driver
         .find_elements_by_xpath("//div[@class='CategoryTreeItem']"))

while nodos != []:
    
    time.sleep(0.2) # Wikipedia tarda en contestar
    nombre_mitologia = nodos[0].text[:(nodos[0].text.find("(") - 1)]
    
    # si no es mitología o ya está en la lista...
    if (not(es_mitologia(nombre_mitologia)) or en_lista(nombre_mitologia, mitologias2)):
        del nodos[0] # se elimina el nodo
        continue
    
    # si es mitología y no está en la lista...
    mitologias2.append(nombre_mitologia) # se añade a la lista y...
    if expand_mitologia(nombre_mitologia): # se despliega el nodo si es posible
        (driver
         .find_element_by_xpath("//span[@data-ct-title='{}']"
                                .format(nombre_mitologia)
                                .replace(" ", "_"))
         .click())
        nodos = (driver
                 .find_elements_by_xpath("//div[@class='CategoryTreeItem']"))
        
driver.close()

Algunas categorías genéricas contienen " por ", por ejemplo, "Mitologías por región". Así que se eliminan de la lista.

In [13]:
mitologias2 = list(compress(mitologias2, [" por " not in m for m in mitologias2]))

In [14]:
mitologias2

['Mitología de África',
 'Mitología bereber',
 'Mitología cartaginesa',
 'Mitología egipcia',
 'Mitología guanche',
 'Mitología igbo',
 'Mitología kushita',
 'Mitología malgache',
 'Mitología masai',
 'Mitología pigmea',
 'Mitología yoruba',
 'Mitología zulú',
 'Mitología de América',
 'Mitología del Caribe',
 'Mitología de América Central',
 'Mitología anishinaabe',
 'Mitología de América del Norte',
 'Mitología afroamericana',
 'Mitología de Costa Rica',
 'Mitología de El Salvador',
 'Mitología talamanqueña',
 'Mitología de Guatemala',
 'Mitología bribri',
 'Mitología mesoamericana',
 'Mitología taína',
 'Mitología canadiense',
 'Mitología mexica',
 'Mitología maya',
 'Mitología tlaxcalteca',
 'Mitología mixteca',
 'Mitología olmeca',
 'Mitología purépecha',
 'Mitología tolteca',
 'Mitología inuit',
 'Mitología circumpolar',
 'Mitología estadounidense',
 'Mitología ainu',
 'Mitología hopi',
 'Mitología pueblo',
 'Mitología mexicana',
 'Mitología de América del Sur',
 'Mitología de As

#### Mitologías 3

Se usa *BeautifulSoup*.

In [16]:
url = "https://es.wikipedia.org/wiki/Anexo:Deidades"

nodos = get_url(url).find("div", id = "toc")
items = [item.text for item in nodos.find_all("span", class_ = "toctext")]

mitologias3 = []
for item in items:
    if item[:10] == "Mitología ":
        mitologias3.append(item)

In [17]:
mitologias3

['Mitología abenaki',
 'Mitología aborigen australiana',
 'Mitología judaica',
 'Mitología del cristianismo',
 'Mitología del catolicismo',
 'Mitología akamba',
 'Mitología akan',
 'Mitología alur',
 'Mitología árabe',
 'Mitología ashanti',
 'Mitología azteca',
 'Mitología baganda',
 'Mitología banyarwanda',
 'Mitología budista',
 'Mitología bushongo',
 'Mitología celta',
 'Mitología china',
 'Mitología chippewa',
 'Mitología coreana',
 'Mitología creek',
 'Mitología dacia',
 'Mitología dahomey',
 'Mitología dinka',
 'Mitología efik',
 'Mitología egipcia',
 'Mitología escandinava',
 'Mitología etrusca',
 'Mitología finlandesa',
 'Mitología frigia',
 'Mitología griega',
 'Mitología guanche',
 'Mitología guaraní',
 'Mitología haida',
 'Mitología hinduista',
 'Mitología hopi',
 'Mitología huron',
 'Mitología igbo',
 'Mitología inca',
 'Mitología inuit',
 'Mitología iroquesa',
 'Mitología isoko',
 'Mitología japonesa',
 'Mitología khoikhoi',
 'Mitología kongo',
 'Mitología kurumba',
 'Mito

#### Mitologías 4

Las anteriores listas no recogen todas las mitologías de Wikipedia. Se añaden 13 mitologías fuera de la lista.

In [18]:
mitologias4 = ["Mitología africana", "Mitología americana", "Mitología asiática", "Mitología europea", "Mitología oceánica",
               "Mitología semítica", "Mitología nubia", "Mitología del hinduismo", "Mitología judía", "Mitología vudú",
               "Mitología hawaiana", "Mitología samoana", "Mitología tahitiana"]

#### Agregación de las mitologías

In [23]:
lista_mitologias = list(pd.unique(mitologias1 + mitologias2 + mitologias3 + mitologias4))

lista_mitologias

['Mitología afroamericana',
 'Mitología amazónica',
 'Mitología de América del Norte',
 'Mitología árabe',
 'Mitología aborigen australiana',
 'Mitología bereber',
 'Mitología cananea',
 'Mitología cartaginesa',
 'Mitología celta',
 'Mitología cristiana',
 'Mitología egipcia',
 'Mitología eslava',
 'Mitología etrusca',
 'Mitología finlandesa',
 'Mitología germana',
 'Mitología griega',
 'Mitología guanche',
 'Mitología hinduista',
 'Mitología hopi',
 'Mitología igbo',
 'Mitología incaica',
 'Mitología inuit',
 'Mitología irania',
 'Mitología irlandesa',
 'Mitología itálica',
 'Mitología del judaísmo',
 'Mitología kushita',
 'Mitología malgache',
 'Mitología maorí',
 'Mitología mapuche',
 'Mitología masai',
 'Mitología mesoamericana',
 'Mitología mesopotámica',
 'Mitología íbera',
 'Mitología lituana',
 'Mitología maya',
 'Mitología mexica',
 'Mitología pigmea',
 'Mitología pueblo',
 'Mitología romana',
 'Mitología selknam',
 'Mitología taína',
 'Mitología túrquica',
 'Mitología venezol

### Mitologías por región

Se pueden encontrar algunas mitologías en inglés con sus respectivas regiones en el siguiente [link](https://en.wikipedia.org/wiki/List_of_mythologies). Las mitologías restantes sin región se añaden a mano.

#### Obtención en inglés

Se usa *BeautifulSoup*.

Primero se obtienen los elementos de la URL.

In [38]:
url = "https://en.wikipedia.org/wiki/List_of_mythologies"

items = get_url(url).find("div", class_ = "mw-parser-output").contents

items_clean = []
for item in items:
    if not(isinstance(item, str)) and item.name in ["h2", "h3", "h4", "ul"]:
        items_clean.append(item)

del items_clean[0]

In [39]:
# Función para limpiar el texto de la region

def clean_edit(string):
    if string[-6:] == "[edit]":
        return(string[:-6])
    else:
        return(string)

Se recorren los elementos, clasificando las mitologías en su región.

In [40]:
mythologies = pd.DataFrame()
h2 = True
i = 0
while h2:
    if items_clean[i].name == "h2":
        h2 = False
    elif items_clean[i].name == "h3":
        h3 = clean_edit(items_clean[i].text)
        h4 = None
    elif items_clean[i].name == "h4":
        h4 = clean_edit(items_clean[i].text)
    elif items_clean[i].name == "ul":
        myth_split = items_clean[i].text.split("\n")
        for item in myth_split:
            if item[-9:] == "mythology":
                mythologies = mythologies.append(pd.DataFrame([[item, h4 if h4 is not None else h3, h3]]))
    i += 1

mythologies = mythologies.reset_index(drop = True)
mythologies.columns = ["Mythology", "Region", "Continent"]

mythologies

,Mythology,Region,Continent
0,Baluba mythology,Central Africa,Africa
1,Bushongo mythology,Central Africa,Africa
2,Kongo mythology,Central Africa,Africa
3,Lugbara mythology,Central Africa,Africa
4,Mbuti mythology,Central Africa,Africa
...,...,...,...
142,Chilote mythology,South America,Americas
143,Guarani mythology,South America,Americas
144,Inca mythology,South America,Americas
145,Mapuche mythology,South America,Americas


#### Unión en español

Accedemos a las páginas de las mitologías anteriores en inglés y buscamos si existen los artículos en español. En caso afirmativo, obtenemos el nombre de la mitología en español y actualizamos la tabla anterior con las regiones.

In [44]:
mitologias_esp = [None] * len(mythologies)
url_base = "https://en.wikipedia.org/wiki/{}"
for i in range(len(mythologies)):
    url = url_base.format(mythologies["Mythology"][i].replace(" ", "_"))
    page = get_url(url)
    if page is None:
        continue
    es_link = page.find("a", hreflang = "es", class_ = "interlanguage-link-target")
    if es_link is not None:
        pos = es_link.get("title").find(" – Spanish")
        mitologias_esp[i] = es_link.get("title")[:pos]

mythologies["Mitología"] = mitologias_esp

del mythologies["Mythology"]

mythologies = mythologies[mythologies["Mitología"].isin(lista_mitologias)]

In [45]:
mythologies

,Region,Continent,Mitología
3,Central Africa,Africa,Mitología lugbara
11,North Africa,Africa,Mitología egipcia
28,Northeast Asia,Asia,Mitología china
29,Northeast Asia,Asia,Mitología japonesa
30,Northeast Asia,Asia,Mitología coreana
37,Northeast Asia,Asia,Mitología túrquica
38,South Asia,Asia,Mitología budista
39,South Asia,Asia,Mitología hinduista
53,West Asia / Middle East,Asia,Mitología cristiana
55,West Asia / Middle East,Asia,Mitología judía


Falta añadir la región de las mitologías siguientes. A pesar de que sean muchas, considero que la región es una variable suficientemente importante para añadirla a mano.

In [46]:
set(lista_mitologias) - set(mythologies["Mitología"])

{'Mitología Báltica',
 'Mitología abenaki',
 'Mitología abrahámica',
 'Mitología acadia',
 'Mitología africana',
 'Mitología afroamericana',
 'Mitología aimara',
 'Mitología ainu',
 'Mitología akamba',
 'Mitología akan',
 'Mitología alur',
 'Mitología amazónica',
 'Mitología americana',
 'Mitología anatolia',
 'Mitología anglosajona',
 'Mitología anishinaabe',
 'Mitología aragonesa',
 'Mitología argentina',
 'Mitología ashanti',
 'Mitología asiria',
 'Mitología asiática',
 'Mitología asturiana',
 'Mitología azteca',
 'Mitología babilónica',
 'Mitología baganda',
 'Mitología banyarwanda',
 'Mitología bereber',
 'Mitología boliviana',
 'Mitología bribri',
 'Mitología bushongo',
 'Mitología canadiense',
 'Mitología cananea',
 'Mitología cartaginesa',
 'Mitología castellana',
 'Mitología catalana',
 'Mitología chibcha',
 'Mitología chilena',
 'Mitología chippewa',
 'Mitología circumpolar',
 'Mitología colombiana',
 'Mitología dacia',
 'Mitología dahomey',
 'Mitología de América',
 'Mitolog

In [47]:
addMyths = [["Northern Europe", "Europe", "Mitología Báltica"],
            ["North America", "Americas", "Mitología abenaki"],
            ["West Asia / Middle East", "Asia", "Mitología abrahámica"],
            ["West Asia / Middle East", "Asia", "Mitología acadia"],
            ["Africa", "Africa", "Mitología africana"],
            ["African Diasporic", "Africa", "Mitología afroamericana"],
            ["South America", "Americas", "Mitología aimara"],
            ["North America", "Americas", "Mitología ainu"],
            ["East Africa", "Africa", "Mitología akamba"],
            ["West Africa", "Africa", "Mitología akan"],
            ["Central Africa", "Africa", "Mitología alur"],
            ["South America", "Americas", "Mitología amazónica"],
            ["Americas", "Americas", "Mitología americana"],
            ["West Asia / Middle East", "Asia", "Mitología anatolia"],
            ["Northern Europe", "Europe", "Mitología anglosajona"],
            ["Central America", "Americas", "Mitología anishinaabe"],
            ["Western Europe", "Europe", "Mitología aragonesa"],
            ["South America", "Americas", "Mitología argentina"],
            ["West Africa", "Africa", "Mitología ashanti"],
            ["West Asia / Middle East", "Asia", "Mitología asiria"],
            ["Asia", "Asia", "Mitología asiática"],
            ["Western Europe", "Europe", "Mitología asturiana"],
            ["Central America", "Americas", "Mitología azteca"],
            ["West Asia / Middle East", "Asia", "Mitología babilónica"],
            ["East Africa", "Africa", "Mitología baganda"],
            ["East Africa", "Africa", "Mitología banyarwanda"],
            ["North Africa", "Africa", "Mitología bereber"],
            ["South America", "Americas", "Mitología boliviana"],
            ["Central America", "Americas", "Mitología bribri"],
            ["Central Africa", "Africa", "Mitología bushongo"],
            ["North America", "Americas", "Mitología canadiense"],
            ["West Asia / Middle East", "Asia", "Mitología cananea"],
            ["North Africa", "Africa", "Mitología cartaginesa"],
            ["Western Europe", "Europe", "Mitología castellana"],
            ["Western Europe", "Europe", "Mitología catalana"],
            ["South America", "Americas", "Mitología chibcha"],
            ["South America", "Americas", "Mitología chilena"],
            ["North America", "Americas", "Mitología chippewa"],
            ["North America", "Americas", "Mitología circumpolar"],
            ["South America", "Americas", "Mitología colombiana"],
            ["Eastern Europe", "Europe", "Mitología dacia"],
            ["West Africa", "Africa", "Mitología dahomey"],
            ["Americas", "Americas", "Mitología de América"],
            ["Central America", "Americas", "Mitología de América Central"],
            ["North America", "Americas", "Mitología de América del Norte"],
            ["South America", "Americas", "Mitología de América del Sur"],
            ["Southern Europe", "Europe", "Mitología de Argólida"],
            ["Asia", "Asia", "Mitología de Asia"],
            ["Northeast Asia", "Asia", "Mitología de Asia Oriental"],
            ["Central America", "Americas", "Mitología de Costa Rica"],
            ["Southern Europe", "Europe", "Mitología de Creta"],
            ["Central America", "Americas", "Mitología de El Salvador"],
            ["Europe", "Europe", "Mitología de Europa"],
            ["Oceania", "Oceania", "Mitología de Fiyi"],
            ["Western Europe", "Europe", "Mitología de Galicia"],
            ["Southern Europe", "Europe", "Mitología de Grecia Central"],
            ["Central America", "Americas", "Mitología de Guatemala"],
            ["Eastern Europe", "Europe", "Mitología de Hungría"],
            ["West Asia / Middle East", "Asia", "Mitología de Levante"],
            ["Oceania", "Oceania", "Mitología de Melanesia"],
            ["Oceania", "Oceania", "Mitología de Micronesia"],
            ["Oceania", "Oceania", "Mitología de Oceanía"],
            ["West Asia / Middle East", "Asia", "Mitología de Oriente Medio"],
            ["Oceania", "Oceania", "Mitología de Polinesia"],
            ["Eastern Europe", "Europe", "Mitología de Rumania"],
            ["Africa", "Africa", "Mitología de África"],
            ["Southern Europe", "Europe", "Mitología de Élide"],
            ["West Asia / Middle East", "Asia", "Mitología del Antiguo Oriente Próximo"],
            ["Central Asia", "Asia", "Mitología del Asia Central"],
            ["South Asia", "Asia", "Mitología del Asia meridional"],
            ["Northeast Asia", "Asia", "Mitología del Asia norte"],
            ["Southeast Asia", "Asia", "Mitología del Asia sureste"],
            ["Central America", "Americas", "Mitología del Caribe"],
            ["Western Europe", "Europe", "Mitología del País Vasco"],
            ["Southern Europe", "Europe", "Mitología del Peloponeso"],
            ["Western Europe", "Europe", "Mitología del catolicismo"],
            ["West Asia / Middle East", "Asia", "Mitología del cristianismo"],
            ["South Asia", "Asia", "Mitología del hinduismo"],
            ["West Asia / Middle East", "Asia", "Mitología del islam"],
            ["West Asia / Middle East", "Asia", "Mitología del judaísmo"],
            ["North Africa", "Africa", "Mitología dinka"],
            ["Southern Europe", "Europe", "Mitología doriana"],
            ["West Africa", "Africa", "Mitología efik"],
            ["Northern Europe", "Europe", "Mitología escandinava"],
            ["Eastern Europe", "Europe", "Mitología eslava"],
            ["Western Europe", "Europe", "Mitología española"],
            ["North America", "Americas", "Mitología estadounidense"],
            ["Europe", "Europe", "Mitología europea"],
            ["Western Europe", "Europe", "Mitología extremeña"],
            ["West Asia / Middle East", "Asia", "Mitología fenicia"],
            ["Southeast Asia", "Asia", "Mitología filipina"],
            ["West Asia / Middle East", "Asia", "Mitología filistea"],
            ["Northern Europe", "Europe", "Mitología finlandesa"],
            ["Western Europe", "Europe", "Mitología francesa"],
            ["West Asia / Middle East", "Asia", "Mitología frigia"],
            ["Western Europe", "Europe", "Mitología gala"],
            ["West Asia / Middle East", "Asia", "Mitología griega de Asia Menor"],
            ["Southern Europe", "Europe", "Mitología griega de Chipre"],
            ["Southern Europe", "Europe", "Mitología griega de Corinto"],
            ["Southern Europe", "Europe", "Mitología griega de Sicilia"],
            ["Southern Europe", "Europe", "Mitología griega de Tesalia"],
            ["Southern Europe", "Europe", "Mitología griega de Tracia"],
            ["Southern Europe", "Europe", "Mitología griega de Épiro"],
            ["West Africa", "Africa", "Mitología guanche"],
            ["North America", "Americas", "Mitología haida"],
            ["West Asia / Middle East", "Asia", "Mitología hattiana"],
            ["Oceania", "Oceania", "Mitología hawaiana"],
            ["South America", "Americas", "Mitología huilliche"],
            ["North America", "Americas", "Mitología huron"],
            ["West Asia / Middle East", "Asia", "Mitología hurrita"],
            ["West Africa", "Africa", "Mitología igbo"],
            ["South America", "Americas", "Mitología inca"],
            ["Eastern Europe", "Europe", "Mitología indoeuropea"],
            ["North America", "Americas", "Mitología iroquesa"],
            ["West Africa", "Africa", "Mitología isoko"],
            ["Southern Europe", "Europe", "Mitología itálica"],
            ["West Asia / Middle East", "Asia", "Mitología judaica"],
            ["Southern Africa", "Africa", "Mitología khoikhoi"],
            ["Central Africa", "Africa", "Mitología kongo"],
            ["West Africa", "Africa", "Mitología kurumba"],
            ["North Africa", "Africa", "Mitología kushita"],
            ["North America", "Americas", "Mitología kwakiutl"],
            ["Southern Europe", "Europe", "Mitología laconia"],
            ["North America", "Americas", "Mitología lakota"],
            ["Northern Europe", "Europe", "Mitología lapona"],
            ["Northern Europe", "Europe", "Mitología letona"],
            ["West Asia / Middle East", "Asia", "Mitología levantina"],
            ["North Africa", "Africa", "Mitología lotuko"],
            ["East Africa", "Africa", "Mitología malgache"],
            ["South America", "Americas", "Mitología mapuche"],
            ["East Africa", "Africa", "Mitología masai"],
            ["Central America", "Americas", "Mitología mesoamericana"],
            ["West Asia / Middle East", "Asia", "Mitología mesopotámica"],
            ["Central America", "Americas", "Mitología mexicana"],
            ["Central America", "Americas", "Mitología mixteca"],
            ["North America", "Americas", "Mitología navajo"],
            ["North Africa", "Africa", "Mitología nubia"],
            ["Oceania", "Oceania", "Mitología oceánica"],
            ["North America", "Americas", "Mitología pawnee"],
            ["South America", "Americas", "Mitología peruana"],
            ["Central Africa", "Africa", "Mitología pigmea"],
            ["Western Europe", "Europe", "Mitología portuguesa"],
            ["North America", "Americas", "Mitología pueblo"],
            ["Central America", "Americas", "Mitología purépecha"],
            ["South America", "Americas", "Mitología quechua"],
            ["Eastern Europe", "Europe", "Mitología rusa"],
            ["Northern Europe", "Europe", "Mitología sajona y anglosajona"],
            ["North America", "Americas", "Mitología salish"],
            ["Southern Europe", "Europe", "Mitología sarda"],
            ["South America", "Americas", "Mitología selknam"],
            ["West Asia / Middle East", "Asia", "Mitología semítica"],
            ["Northeast Asia", "Asia", "Mitología shinto"],
            ["Northeast Asia", "Asia", "Mitología siberiana"],
            ["West Asia / Middle East", "Asia", "Mitología sumeria"],
            ["North America", "Americas", "Mitología séneca"],
            ["Oceania", "Oceania", "Mitología tahitiana"],
            ["Central America", "Americas", "Mitología taína"],
            ["Central America", "Americas", "Mitología tlaxcalteca"],
            ["Central America", "Americas", "Mitología tolteca"],
            ["Southern Europe", "Europe", "Mitología tracia"],
            ["East Africa", "Africa", "Mitología tumbuka"],
            ["Eastern Europe", "Europe", "Mitología ucraniana"],
            ["West Asia / Middle East", "Asia", "Mitología ugaritiche"],
            ["South America", "Americas", "Mitología venezolana"],
            ["African Diasporic", "Africa", "Mitología vudú"],
            ["North America", "Americas", "Mitología winnebago"],
            ["Central America", "Americas", "Mitología wixarrica"],
            ["West Africa", "Africa", "Mitología yoruba"],
            ["Southern Africa", "Africa", "Mitología zulú"],
            ["North America", "Americas", "Mitología zuñi"],
            ["Southern Europe", "Europe", "Mitología ática"],
            ["Western Europe", "Europe", "Mitología íbera"]]

In [55]:
mitologias = (mythologies
              .append(pd.DataFrame(addMyths, columns = ["Region", "Continent", "Mitología"]))
              .drop_duplicates()
              .set_index("Mitología"))

mitologias.columns = ["Región", "Continente"]

mitologias

,Región,Continente
Mitología,,
Mitología lugbara,Central Africa,Africa
Mitología egipcia,North Africa,Africa
Mitología china,Northeast Asia,Asia
Mitología japonesa,Northeast Asia,Asia
Mitología coreana,Northeast Asia,Asia
...,...,...
Mitología yoruba,West Africa,Africa
Mitología zulú,Southern Africa,Africa
Mitología zuñi,North America,Americas


In [56]:
set(mitologias["Región"])

{'Africa',
 'African Diasporic',
 'Americas',
 'Asia',
 'Central Africa',
 'Central America',
 'Central Asia',
 'East Africa',
 'Eastern Europe',
 'Europe',
 'North Africa',
 'North America',
 'Northeast Asia',
 'Northern Europe',
 'Oceania',
 'South America',
 'South Asia',
 'Southeast Asia',
 'Southern Africa',
 'Southern Europe',
 'West Africa',
 'West Asia / Middle East',
 'Western Europe'}

Se cambian los nombres de las regiones y los continentes al español.

In [65]:
region_esp = {"Africa" : "África",
              "African Diasporic" : "Diáspora Africana",
              "Americas" : "América",
              #"Asia" : "Asia",
              "Central Africa" : "África Central",
              "Central America" : "América Central",
              "Central Asia" : "Asia Central",
              "East Africa" : "África del Este",
              "Eastern Europe" : "Europa del Este",
              "Europe" : "Europa",
              "North Africa" : "África del Norte",
              "North America" : "América del Norte",
              "Northeast Asia" : "Asia del Noreste",
              "Northern Europe" : "Europa del Norte",
              "Oceania" : "Oceanía",
              "South America" : "América del Sur",
              "South Asia" : "Asia del Sur",
              "Southeast Asia" : "Asia del Sureste",
              "Southern Africa" : "África del Sur",
              "Southern Europe" : "Europa del Sur",
              "West Africa" : "África del Oeste",
              "West Asia / Middle East" : "Oriente Medio",
              "Western Europe" : "Europa del Oeste"}

mitologias = mitologias.replace({"Región" : region_esp, "Continente" : region_esp})
mitologias

,Región,Continente
Mitología,,
Mitología lugbara,África Central,África
Mitología egipcia,África del Norte,África
Mitología china,Asia del Noreste,Asia
Mitología japonesa,Asia del Noreste,Asia
Mitología coreana,Asia del Noreste,Asia
...,...,...
Mitología yoruba,África del Oeste,África
Mitología zulú,África del Sur,África
Mitología zuñi,América del Norte,América


In [67]:
mitologias.to_csv("../data/mitologias.csv", header = True, index = True)

## 2. Personajes por categorías específicas

Con la lista de mitologías, se buscan las categorías disponibles en Wikipedia que únicamente contengan personajes mitológicos. Algunas categorías son únicas para cada mitología en particular, así que se añaden a mano.  
De esas categorías, se obtienen todos los personajes.

In [3]:
mitologias = pd.read_csv("../data/mitologias.csv")
mitologias

,Mitología,Región,Continente
0,Mitología lugbara,África Central,África
1,Mitología egipcia,África del Norte,África
2,Mitología china,Asia del Noreste,Asia
3,Mitología japonesa,Asia del Noreste,Asia
4,Mitología coreana,Asia del Noreste,Asia
...,...,...,...
210,Mitología yoruba,África del Oeste,África
211,Mitología zulú,África del Sur,África
212,Mitología zuñi,América del Norte,América
213,Mitología ática,Europa del Sur,Europa


### Obtención de las URLs de las categorías

Primero se obtiene un dataset con todas las URLs de las categorías que scrapearemos más adelante.

Las URLs de las categorías genéricas están formadas por (1) el *tipo de categoría* y (2) la *mitología*.

Ejemplo: `https://es.wikipedia.org/wiki/Categoría:Dioses_griegos` , donde `Dioses` es la categoría y `griegos` es la mitología.

1. Existen diferentes categorías que contienen personajes mitológicos (Deidades, Dioses, Gigantes, Demonios, Dragones ...).  
    Cada mitología tiene sus propias categorías, por lo que he elegido las categorías que he considerado compartidas entre mitologías. Estas categorías son: Personaje, Deidad, Dios, Diosa, Criatura, Dragón, Gigante, Demonio, Monstruo y Héroe.  
    Por otro lado, cada categoría puede tener diversos formatos según la mitología en particular. Por ejemplo, la categoría de "Criatura" puede estar formateada como: `Criaturas`, `Criaturas_mitológicas` y `Criaturas_de_la_mitología`.

In [4]:
# anteriormente se han probado más formatos
categs = {"Personajes_de_la_mitología" : "Personaje", "Deidades" : "Deidad", "Dioses_de_la_mitología" : "Dios",
          "Dioses" : "Dios", "Diosas_de_la_mitología" : "Diosa", "Diosas" : "Diosa", "Criaturas_de_la_mitología" : "Criatura",
          "Criaturas_mitológicas" : "Criatura", "Dragones" : "Dragón", "Gigantes_de_la_mitología" : "Gigante"}

2. El formato de la mitología depende tanto del tipo/formato de la categoría como de la propia mitología.  
    Por ejemplo, en la mitología egipcia podemos tener `Dioses_egipcios`, `Diosas_egipcias` y `Deidades_de_la_mitología_egipcia`. En cada caso, el formato de la mitología es diferente.  
    Además, cada mitología tiene su particular transformación cuando cambia de género y número. Los sustantivos del español suelen seguir unas reglas para esta transformación, aunque existen excepciones. Se aplican las 5 reglas más comunes a las mitologías para obtener sus términos clave, por lo que probablemente una de estas claves será correcta.  
    Algunas URLs tienen mayúscula la primera letra de la mitología, por lo que añado estas posibilidades a las claves.

In [5]:
# Funcion para obtener los terminos clave de una mitologia

def clave_mitologia(mit):
    lista = []
    lista.append("_".join(mit.split()[1:])) # elemento 0 en minuscula
    lista.append("_".join(mit.split()[1:-1] + [mit.split()[-1].capitalize()])) # elemento 1 en mayuscula
    for i in range(2):
        lista.append(lista[i] + "s")
        lista.append(lista[i] + "es")
        lista.append(lista[i][0:-1] + "os")
        lista.append(lista[i][0:-1] + "es")
    return(lista)

In [6]:
# Funcion para comprobar si existe la URL de la categoria

def url_buena(url):
    doc = get_url(url)
    if doc != None:
        paginas = doc.find("div", id = "mw-pages")
        if paginas != None:
            return(True)
    return(False)

In [7]:
# Funcion para obtener las posibles URLs de una mitologia

def obtener_urls(mit):
    url_base = "https://es.wikipedia.org/wiki/Categoría:{}_"
    urls = pd.DataFrame()
    keys = clave_mitologia(mit)
    formatos = list(categs.keys())
    for formato in formatos:
        key_found = False
        i = 0
        while (not(key_found) and i < 10):
            url = url_base.format(formato) + keys[i]
            if url_buena(url):
                fila = pd.DataFrame([[url,
                                      mit,
                                      categs[formato]]])
                urls = urls.append(fila)
                key_found = True
            i += 1
    return(urls.drop_duplicates(ignore_index = True))

**Tarda mucho en ejecutarse**  
Se combinan los tipos de categoría y las mitologías y se obtiene un dataset con las categorías existentes en Wikipedia.

In [8]:
urls_categorias = pd.DataFrame()

for mitologia in mitologias["Mitología"]:
    urls_categorias = urls_categorias.append(obtener_urls(mitologia))

urls_categorias.columns = ["URL", "Mitología", "Clase"]

In [9]:
urls_categorias

,URL,Mitología,Clase
0,https://es.wikipedia.org/wiki/Categoría:Dioses...,Mitología egipcia,Dios
1,https://es.wikipedia.org/wiki/Categoría:Diosas...,Mitología egipcia,Diosa
2,https://es.wikipedia.org/wiki/Categoría:Criatu...,Mitología egipcia,Criatura
0,https://es.wikipedia.org/wiki/Categoría:Dioses...,Mitología china,Dios
1,https://es.wikipedia.org/wiki/Categoría:Diosas...,Mitología china,Diosa
...,...,...,...
0,https://es.wikipedia.org/wiki/Categoría:Dioses...,Mitología tracia,Dios
1,https://es.wikipedia.org/wiki/Categoría:Diosas...,Mitología tracia,Diosa
0,https://es.wikipedia.org/wiki/Categoría:Criatu...,Mitología ucraniana,Criatura
0,https://es.wikipedia.org/wiki/Categoría:Dioses...,Mitología vudú,Dios


Se añaden directamente las siguientes categorías con formato único:
* [Dioses primordiales de la mitología griega](https://es.wikipedia.org/wiki/Categoría:Dioses_primordiales_de_la_mitología_griega)
* [Titanes (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Titanes)
* [Deidades de la mitología griega](https://es.wikipedia.org/wiki/Categoría:Deidades_de_la_mitología_griega)
* [Dioses olímpicos menores de la mitología griega](https://es.wikipedia.org/wiki/Categoría:Dioses_olímpicos_menores)
* [Héroes de la mitología griega](https://es.wikipedia.org/wiki/Categoría:Héroes_de_la_mitología_griega)
* [Monstruos de la mitología griega](https://es.wikipedia.org/wiki/Categoría:Monstruos_de_la_mitología_griega)
* [Musas (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Musas)
* [Oceánidas (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Oceánidas)
* [Oceánides (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Oceánides)
* [Descendientes de Zeus (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Zeus)
* [Descendientes de Poseidón (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Poseidón)
* [Descendientes de Apolo (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Apolo)
* [Descendientes de Afrodita (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Afrodita)
* [Diosas tomadas por Zeus (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Diosas_tomadas_por_Zeus)
* [Diosas amadas por Apolo (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Diosas_amadas_por_Apolo)
* [Dioses que tomaron a Afrodita (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Dioses_que_tomaron_a_Afrodita)
* [Jotuns (mitología nórdica)](https://es.wikipedia.org/wiki/Categoría:Jotuns)
* [Vanir (mitología nórdica)](https://es.wikipedia.org/wiki/Categoría:Vanir)
* [Æsir (mitología nórdica)](https://es.wikipedia.org/wiki/Categoría:Æsir)
* [Ásynjur (mitología nórdica)](https://es.wikipedia.org/wiki/Categoría:Ásynjur)
* [Nornas (mitología nórdica)](https://es.wikipedia.org/wiki/Categoría:Nornas)
* [Lobos de la mitología nórdica](https://es.wikipedia.org/wiki/Categoría:Lobos_de_la_mitología_nórdica)
* [Hijos de Odín (mitología nórdica)](https://es.wikipedia.org/wiki/Categoría:Hijos_de_Odín)
* [Demonios de la mitología hinduista](https://es.wikipedia.org/wiki/Categoría:Demonios_en_el_hinduismo)
* [Divinidades sintoistas (mitología japonesa)](https://es.wikipedia.org/wiki/Categoría:Divinidades_sintoistas)
* [Yōkai (mitología japonesa)](https://es.wikipedia.org/wiki/Categoría:Yōkai)
* [Tonalteteuctin (mitología mexica)](https://es.wikipedia.org/wiki/Categoría:Tonalteteuctin)
* [Dioses del viento de la mitología mexica](https://es.wikipedia.org/wiki/Categoría:Dioses_del_viento_de_la_mitología_mexica)
* [Criaturas de la mitología mexica](https://es.wikipedia.org/wiki/Categoría:Criaturas_mexicas)
* [Demonios de la mitología mexica](https://es.wikipedia.org/wiki/Categoría:Demonios_de_la_mitología_mexica)
* [Dharmapalas (mitología budista)](https://es.wikipedia.org/wiki/Categoría:Dharmapalas)
* [Dakinis (mitología budista)](https://es.wikipedia.org/wiki/Categoría:Dakinis)
* [Deidades del budismo tibetano](https://es.wikipedia.org/wiki/Categoría:Deidades_del_budismo_tibetano)
* [Seres de la mitología budista](https://es.wikipedia.org/wiki/Categoría:Seres_de_la_mitología_budista)
* [Demonios de la mitología budista](https://es.wikipedia.org/wiki/Categoría:Demonios_budistas)
* [Ráksasas de la mitología del hinduismo y budista](https://es.wikipedia.org/wiki/Categoría:Ráksasas)

In [10]:
addCategs = [["https://es.wikipedia.org/wiki/Categoría:Dioses_primordiales_de_la_mitología_griega","Mitología griega","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Titanes","Mitología griega","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Deidades_de_la_mitología_griega","Mitología griega","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Dioses_olímpicos_menores","Mitología griega","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Héroes_de_la_mitología_griega","Mitología griega","Héroe"],
             ["https://es.wikipedia.org/wiki/Categoría:Monstruos_de_la_mitología_griega","Mitología griega","Monstruo"],
             ["https://es.wikipedia.org/wiki/Categoría:Musas","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Oceánidas","Mitología griega","Dios"],
             ["https://es.wikipedia.org/wiki/Categoría:Oceánides","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Zeus","Mitología griega","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Poseidón","Mitología griega","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Apolo","Mitología griega","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Afrodita","Mitología griega","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Diosas_tomadas_por_Zeus","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Diosas_amadas_por_Apolo","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Dioses_que_tomaron_a_Afrodita","Mitología griega","Dios"],
             ["https://es.wikipedia.org/wiki/Categoría:Jotuns","Mitología nórdica","Gigante"],
             ["https://es.wikipedia.org/wiki/Categoría:Vanir","Mitología nórdica","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Æsir","Mitología nórdica","Dios"],
             ["https://es.wikipedia.org/wiki/Categoría:Ásynjur","Mitología nórdica","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Nornas","Mitología nórdica","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Lobos_de_la_mitología_nórdica","Mitología nórdica","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Hijos_de_Odín","Mitología nórdica","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Demonios_en_el_hinduismo","Mitología del hinduismo","Demonio"],
             ["https://es.wikipedia.org/wiki/Categoría:Divinidades_sintoistas","Mitología japonesa","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Yōkai","Mitología japonesa","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Tonalteteuctin","Mitología mexica","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Dioses_del_viento_de_la_mitología_mexica","Mitología mexica","Dios"],
             ["https://es.wikipedia.org/wiki/Categoría:Criaturas_mexicas","Mitología mexica","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Demonios_de_la_mitología_mexica","Mitología mexica","Demonio"],
             ["https://es.wikipedia.org/wiki/Categoría:Dharmapalas","Mitología budista","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Dakinis","Mitología budista","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Deidades_del_budismo_tibetano","Mitología budista","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Seres_de_la_mitología_budista","Mitología budista","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Demonios_budistas","Mitología budista","Demonio"],
             ["https://es.wikipedia.org/wiki/Categoría:Ráksasas","Mitología del hinduismo","Demonio"],
             ["https://es.wikipedia.org/wiki/Categoría:Ráksasas","Mitología budista","Demonio"],
             ["https://es.wikipedia.org/wiki/Categoría:Orisha","Mitología yoruba","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Tlaloque","Mitología mexica","Dios"],
             ["https://es.wikipedia.org/wiki/Categoría:Centzon_Totochtin","Mitología mexica","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Tres_augustos_y_cinco_emperadores","Mitología china","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Diosas_tomadas_por_Poseidón","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Nereidas","Mitología griega","Deidad"]]

urls_categorias = urls_categorias.append(pd.DataFrame(addCategs, columns = ["URL", "Mitología", "Clase"]))

In [14]:
# Funcion para obtener la URL de la siguiente pagina de personajes en el caso de que exista

def obtener_siguiente(pagina):
    first_link = pagina.find("div", id = "mw-pages").find("a")
    if first_link.text == "página siguiente":
        return "https://es.wikipedia.org" + first_link.get("href")
    elif first_link.text == "página anterior":
        second_link = pagina.find("div", id = "mw-pages").find_all("a")[1]
        if second_link.text == "página siguiente":
            return "https://es.wikipedia.org" + second_link.get("href")

Se añaden las URLs de las categorías con más de una página de personajes.

In [15]:
urls_categorias = urls_categorias.reset_index(drop = True)

for i in range(len(urls_categorias)):
    next_url = obtener_siguiente(get_url(urls_categorias["URL"][i]))
    while next_url is not None:
        new_categoria = pd.DataFrame([[next_url, urls_categorias["Mitología"][i], urls_categorias["Clase"][i]]])
        new_categoria.columns = ["URL", "Mitología", "Clase"]
        urls_categorias = urls_categorias.append(new_categoria)
        next_url = obtener_siguiente(get_url(next_url))

In [16]:
urls_categorias

,URL,Mitología,Clase
0,https://es.wikipedia.org/wiki/Categoría:Dioses...,Mitología egipcia,Dios
1,https://es.wikipedia.org/wiki/Categoría:Diosas...,Mitología egipcia,Diosa
2,https://es.wikipedia.org/wiki/Categoría:Criatu...,Mitología egipcia,Criatura
3,https://es.wikipedia.org/wiki/Categoría:Dioses...,Mitología china,Dios
4,https://es.wikipedia.org/wiki/Categoría:Diosas...,Mitología china,Diosa
...,...,...,...
126,https://es.wikipedia.org/wiki/Categoría:Tres_a...,Mitología china,Personaje
127,https://es.wikipedia.org/wiki/Categoría:Diosas...,Mitología griega,Diosa
128,https://es.wikipedia.org/wiki/Categoría:Nereidas,Mitología griega,Deidad
0,https://es.wikipedia.org/w/index.php?title=Cat...,Mitología griega,Personaje


### Obtención de las URLs de los personajes

Ahora se obtiene un dataset con las URLs de los personajes pertenecientes a las categorías extraídas en el subapartado anterior. También se añaden algunos personajes importantes manualmente.

In [17]:
# Funcion para limpiar el nombre del personaje

def clean_name(nombre):
    if nombre.find("mitología") != -1:
        pos = nombre.find("(")
        return nombre[:pos].strip()
    elif nombre.find("Mitología") != -1:
        pos = nombre.find("(")
        return nombre[:pos].strip()
    elif nombre.find("(dios)") != -1:
        pos = nombre.find("(")
        return nombre[:pos].strip()
    elif nombre.find("deidad") != -1:
        pos = nombre.find("(")
        return nombre[:pos].strip()
    elif nombre.find("personaje") != -1:
        pos = nombre.find("(")
        return nombre[:pos].strip()
    else:
        return(nombre)

Función para obtener los nombres y las URLs de los personajes de la categoría.  
La división alfabética de los personajes puede estar en dos formatos diferentes:  

* Todas las categorias en el mismo nodo
* Cada categoria tiene su nodo

In [18]:
def scrap_categoria(url):
    pagina = get_url(url)
    lista_pers = []
    lista_urls = []
    div_alfabetica = pagina.find("div", id = "mw-pages").find("div", class_ = "mw-content-ltr")
    if div_alfabetica.find("div", class_ = "mw-category") is None: # caso 1
        categorias = div_alfabetica.find_all("h3")
        check_letras = [categoria.text.isalpha() for categoria in categorias]
        personajes = div_alfabetica.find_all("ul")
        for i in range(len(categorias)):
            if check_letras[i]:
                pers = personajes[i]
                lista_pers += [clean_name(per.text) for per in pers]
                lista_urls += ["https://es.wikipedia.org" + per.find("a").get("href") for per in pers]
        return pd.DataFrame({"URL" : lista_urls,
                             "Nombre" : lista_pers,
                             "Mitología" : urls_categorias[urls_categorias["URL"] == url].iloc[0,1],
                             "Clase" : urls_categorias[urls_categorias["URL"] == url].iloc[0,2]})
    else: # caso 2
        categorias = div_alfabetica.find_all("div", class_ = "mw-category-group")
        for categoria in categorias:
            if categoria.find("h3").text.isalpha():
                pers = categoria.find_all("li")
                lista_pers += [clean_name(per.text) for per in pers]
                lista_urls += ["https://es.wikipedia.org" + per.find("a").get("href") for per in pers]
        return pd.DataFrame({"URL" : lista_urls,
                             "Nombre" : lista_pers,
                             "Mitología" : urls_categorias[urls_categorias["URL"] == url].iloc[0,1],
                             "Clase" : urls_categorias[urls_categorias["URL"] == url].iloc[0,2]})

Se obtienen los personajes de las categorías con sus URLs.

In [19]:
urls_personajes1 = pd.DataFrame()

for url in urls_categorias["URL"]:
    urls_personajes1 = urls_personajes1.append(scrap_categoria(url))

In [20]:
urls_personajes1

,URL,Nombre,Mitología,Clase
0,https://es.wikipedia.org/wiki/Aken_(dios),Aken,Mitología egipcia,Dios
1,https://es.wikipedia.org/wiki/Aker,Aker,Mitología egipcia,Dios
2,https://es.wikipedia.org/wiki/Amenhotep_(hijo_...,Amenhotep (hijo de Hapu),Mitología egipcia,Dios
3,https://es.wikipedia.org/wiki/Am%C3%B3n,Amón,Mitología egipcia,Dios
4,https://es.wikipedia.org/wiki/Amset,Amset,Mitología egipcia,Dios
...,...,...,...,...
21,https://es.wikipedia.org/wiki/Yaso_(desambigua...,Yaso (desambiguación),Mitología griega,Personaje
22,https://es.wikipedia.org/wiki/Y%C3%B3bates,Yóbates,Mitología griega,Personaje
23,https://es.wikipedia.org/wiki/Yolao,Yolao,Mitología griega,Personaje
24,https://es.wikipedia.org/wiki/Zacinto_(mitolog...,Zacinto,Mitología griega,Personaje


In [21]:
urls_personajes1.to_csv("urls_pers1.csv", index = False)

## 3. Personajes por intersección de categorías

Ahora se obtendrán los artículos que se encuentren en la categoría de alguna mitología y a su vez en una categoría de personaje mitológico. Por ejemplo: se obtiene el personaje "Fenghuang" porque pertenece a las categorías "Mitología china" y "Aves legendarias".

In [22]:
mitologias["Mitología"]

0       Mitología lugbara
1       Mitología egipcia
2         Mitología china
3      Mitología japonesa
4       Mitología coreana
              ...        
210      Mitología yoruba
211        Mitología zulú
212        Mitología zuñi
213       Mitología ática
214       Mitología íbera
Name: Mitología, Length: 215, dtype: object

Se obtienen las páginas de las categorías disponibles de cada mitología.

In [23]:
lista_url_mits = []

for mitologia in mitologias["Mitología"]:
    url = "https://es.wikipedia.org/wiki/Categoría:" + mitologia.replace(" ","_")
    if url_buena(url):
        lista_url_mits.append(url)
        next_url = obtener_siguiente(get_url(url))
        while next_url is not None:
            lista_url_mits.append(next_url)
            next_url = obtener_siguiente(get_url(next_url))

urls_mits = (pd
             .DataFrame(
                 list(mitologias["Mitología"]),
                 index = ["https://es.wikipedia.org/wiki/Categoría:" + m.replace(" ","_") for m in list(mitologias["Mitología"])])
             .loc[lista_url_mits])
urls_mits.columns = ["Mitología"]

urls_mits

,Mitología
https://es.wikipedia.org/wiki/Categoría:Mitología_egipcia,Mitología egipcia
https://es.wikipedia.org/wiki/Categoría:Mitología_china,Mitología china
https://es.wikipedia.org/wiki/Categoría:Mitología_japonesa,Mitología japonesa
https://es.wikipedia.org/wiki/Categoría:Mitología_coreana,Mitología coreana
https://es.wikipedia.org/wiki/Categoría:Mitología_túrquica,Mitología túrquica
...,...
https://es.wikipedia.org/wiki/Categoría:Mitología_venezolana,Mitología venezolana
https://es.wikipedia.org/wiki/Categoría:Mitología_yoruba,Mitología yoruba
https://es.wikipedia.org/wiki/Categoría:Mitología_zulú,Mitología zulú
https://es.wikipedia.org/wiki/Categoría:Mitología_ática,Mitología ática


Se obtienen todos los artículos de las categorías de mitologías.

In [24]:
lista_urls = []
lista_pers = []
lista_mits = []

for url in urls_mits.index:
    nodos = get_url(url).find("div", id = "mw-pages").find("div", class_ = "mw-content-ltr").find_all("a")
    lista_urls = lista_urls + ["https://es.wikipedia.org" + nodo.get("href") for nodo in nodos]
    lista_pers = lista_pers + [clean_name(nodo.text) for nodo in nodos]
    lista_mits = lista_mits + len(nodos) * [urls_mits.loc[url]["Mitología"]]

articulos_mits = pd.DataFrame({"URL" : lista_urls, "Nombre" : lista_pers, "Mitología" : lista_mits})

articulos_mits

,URL,Nombre,Mitología
0,https://es.wikipedia.org/wiki/Mitolog%C3%ADa_e...,Mitología egipci,Mitología egipcia
1,https://es.wikipedia.org/wiki/Aaru,Aaru,Mitología egipcia
2,https://es.wikipedia.org/wiki/Abatos,Abatos,Mitología egipcia
3,https://es.wikipedia.org/wiki/Aj_(mitolog%C3%ADa),Aj,Mitología egipcia
4,https://es.wikipedia.org/wiki/Amset,Amset,Mitología egipcia
...,...,...,...
2524,https://es.wikipedia.org/wiki/Aletes_(deidad),Aletes,Mitología íbera
2525,https://es.wikipedia.org/wiki/Ataecina,Ataecina,Mitología íbera
2526,https://es.wikipedia.org/wiki/Betatun,Betatun,Mitología íbera
2527,https://es.wikipedia.org/wiki/Necis,Necis,Mitología íbera


In [25]:
# Funcion para obtener las categorias de un articulo

def cats_articulo(url):
    art_cats = get_url(url).find("div", id = "mw-normal-catlinks").find("ul").find_all("a")
    return [cats.text for cats in art_cats]

Funciones para comprobar si los artículos pertenecen a categorías de personajes mitológicos.

In [26]:
def check_deidad(lista):
    check = False
    for l in lista:
        check = any([check, any(x in l for x in ["Deidad", "Dios"])])
    return(check)

In [27]:
def check_criatura(lista):
    check = False
    for l in lista:
        check = any([check, any(x in l for x in ["Criatura", "Animales mitológicos", "Arañas mitológicas", "Aves legendarias",
                                                 "Bovinos mitológicos", "Caballos mitológicos", "Caprinos mitológicos",
                                                 "Felinos mitológicos", "Gatos mitológicos", "Leones mitológicos",
                                                 "Peces mitológicos", "Perros mitológicos", "Reptiles mitológicos",
                                                 "Serpientes mitológicas", "Críptido", "Cambiante", "Quimera"])])
    return(check)

In [28]:
def check_dragon(lista):
    check = False
    for l in lista:
        check = any([check, "Dragones" in l])
    return(check)

In [29]:
def check_gigante(lista):
    check = False
    for l in lista:
        check = any([check, "Gigante" in l])
    return(check)

In [30]:
def check_heroe(lista):
    check = False
    for l in lista:
        check = any([check, "Héroe" in l])
    return(check)

In [31]:
def check_monstruo(lista):
    check = False
    for l in lista:
        check = any([check, "Monstruo" in l])
    return(check)

In [32]:
def check_demonio(lista):
    check = False
    for l in lista:
        check = any([check, "Demonio" in l])
    return(check)

De los artículos anteriores únicamente queremos obtener los personajes mitológicos. Es decir, obtenemos los que pertenecen a categorías que incluyen la palabra "Dios", "Monstruo", "Demonio",...

In [34]:
urls_personajes2 = pd.DataFrame()

for i in range(len(articulos_mits)):
    lista_cats = cats_articulo(articulos_mits["URL"][i])
    if check_deidad(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Deidad"]])))
    if check_criatura(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Criatura"]])))
    if check_dragon(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Dragón"]])))
    if check_gigante(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Gigante"]])))
    if check_heroe(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Héroe"]])))
    if check_monstruo(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Monstruo"]])))
    if check_demonio(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Demonio"]])))

urls_personajes2.columns = ["URL", "Nombre", "Mitología", "Clase"]

In [35]:
urls_personajes2

,URL,Nombre,Mitología,Clase
0,https://es.wikipedia.org/wiki/Amset,Amset,Mitología egipcia,Deidad
0,https://es.wikipedia.org/wiki/Bujis,Bujis,Mitología egipcia,Criatura
0,https://es.wikipedia.org/wiki/Cuervo_de_tres_p...,Cuervo de tres patas,Mitología egipcia,Criatura
0,https://es.wikipedia.org/wiki/Neper_(mitolog%C...,Neper,Mitología egipcia,Deidad
0,https://es.wikipedia.org/wiki/Nun_(mitolog%C3%...,Nun,Mitología egipcia,Deidad
...,...,...,...,...
0,https://es.wikipedia.org/wiki/Aletes_(deidad),Aletes,Mitología íbera,Deidad
0,https://es.wikipedia.org/wiki/Ataecina,Ataecina,Mitología íbera,Deidad
0,https://es.wikipedia.org/wiki/Betatun,Betatun,Mitología íbera,Deidad
0,https://es.wikipedia.org/wiki/Necis,Necis,Mitología íbera,Deidad


In [36]:
urls_personajes2.to_csv("urls_pers2.csv", index = False)

## 4. Dataset de personajes

En este apartado se obtendrá tanto el dataset final con todos los personajes, como la matriz de adyacencia con los links de los mismos personajes.

Primero se unen los dos datasets anteriores.

In [3]:
url_pers = pd.read_csv("urls_pers1.csv").append(pd.read_csv("urls_pers2.csv"))
url_pers

,URL,Nombre,Mitología,Clase
0,https://es.wikipedia.org/wiki/Aken_(dios),Aken,Mitología egipcia,Dios
1,https://es.wikipedia.org/wiki/Aker,Aker,Mitología egipcia,Dios
2,https://es.wikipedia.org/wiki/Amenhotep_(hijo_...,Amenhotep (hijo de Hapu),Mitología egipcia,Dios
3,https://es.wikipedia.org/wiki/Am%C3%B3n,Amón,Mitología egipcia,Dios
4,https://es.wikipedia.org/wiki/Amset,Amset,Mitología egipcia,Dios
...,...,...,...,...
1130,https://es.wikipedia.org/wiki/Aletes_(deidad),Aletes,Mitología íbera,Deidad
1131,https://es.wikipedia.org/wiki/Ataecina,Ataecina,Mitología íbera,Deidad
1132,https://es.wikipedia.org/wiki/Betatun,Betatun,Mitología íbera,Deidad
1133,https://es.wikipedia.org/wiki/Necis,Necis,Mitología íbera,Deidad


Se añaden directamente algunos personajes mitológicos importantes faltantes:

*Mitología griega*
* [Tánatos](https://es.wikipedia.org/wiki/Tánatos)
* [Zeus](https://es.wikipedia.org/wiki/Zeus)
* [Poseidón](https://es.wikipedia.org/wiki/Poseidón)
* [Apolo](https://es.wikipedia.org/wiki/Apolo)
* [Atenea](https://es.wikipedia.org/wiki/Atenea)
* [Artemisa](https://es.wikipedia.org/wiki/Artemisa)
* [Deméter](https://es.wikipedia.org/wiki/Deméter)
* [Heracles](https://es.wikipedia.org/wiki/Heracles)
* [Teseo](https://es.wikipedia.org/wiki/Teseo)
* [Jasón](https://es.wikipedia.org/wiki/Jasón)

*Mitología romana*
* [Mitra](https://es.wikipedia.org/wiki/Mitra_(dios_romano))
* [Magna Mater](https://es.wikipedia.org/wiki/Magna_Mater)

*Mitología nórdica*
* [Hela](https://es.wikipedia.org/wiki/Hela)

*Mitología mesopotámica*
* [Gilgamesh](https://es.wikipedia.org/wiki/Gilgamesh)

*Mitología sumeria*
* [Anunnaki](https://es.wikipedia.org/wiki/Anunnaki)

*Mitología incaica*
* [Cuchavira](https://es.wikipedia.org/wiki/Cuchavira)

*Mitología pueblo*
* [Awelo](https://es.wikipedia.org/wiki/Awelo)

entre otros...

In [4]:
new_pers = [["https://es.wikipedia.org/wiki/Tánatos","Tánatos","Mitología griega","Deidad"],
           ["https://es.wikipedia.org/wiki/Zeus","Zeus","Mitología griega","Dios"],
           ["https://es.wikipedia.org/wiki/Poseidón","Poseidón","Mitología griega","Dios"],
           ["https://es.wikipedia.org/wiki/Apolo","Apolo","Mitología griega","Dios"],
           ["https://es.wikipedia.org/wiki/Atenea","Atenea","Mitología griega","Diosa"],
           ["https://es.wikipedia.org/wiki/Artemisa","Artemisa","Mitología griega","Diosa"],
           ["https://es.wikipedia.org/wiki/Deméter","Deméter","Mitología griega","Diosa"],
           ["https://es.wikipedia.org/wiki/Heracles","Heracles","Mitología griega","Héroe"],
           ["https://es.wikipedia.org/wiki/Teseo","Teseo","Mitología griega","Héroe"],
           ["https://es.wikipedia.org/wiki/Jasón","Jasón","Mitología griega","Héroe"],
           ["https://es.wikipedia.org/wiki/Mitra_(dios_romano)","Mitra","Mitología romana","Dios"],
           ["https://es.wikipedia.org/wiki/Magna_Mater","Magna Mater","Mitología romana","Diosa"],
           ["https://es.wikipedia.org/wiki/Hela","Hela","Mitología nórdica","Deidad"],
           ["https://es.wikipedia.org/wiki/Gilgamesh","Gilgamesh","Mitología mesopotámica","Héroe"],
           ["https://es.wikipedia.org/wiki/Anunnaki", "Anunnaki", "Mitología sumeria", "Deidad"],
           ["https://es.wikipedia.org/wiki/Cuchavira","Cuchavira","Mitología incaica","Dios"],
           ["https://es.wikipedia.org/wiki/Awelo","Awelo","Mitología pueblo","Dios"],
           ["https://es.wikipedia.org/wiki/Asase_Ya","Asase Ya","Mitología africana","Diosa"],
           ["https://es.wikipedia.org/wiki/Xólotl_(divinidad)","Xólotl","Mitología mexica","Dios"],
           ["https://es.wikipedia.org/wiki/Chicomecóatl","Chicomecóatl","Mitología mexica","Diosa"],
           ["https://es.wikipedia.org/wiki/Macuiltochtli","Macuiltochtli","Mitología mexica","Dios"],
           ["https://es.wikipedia.org/wiki/Xochiquétzal","Xochiquétzal","Mitología mexica","Diosa"],
           ["https://es.wikipedia.org/wiki/Ñamandú","Ñamandú","Mitología guaraní","Dios"],
           ["https://es.wikipedia.org/wiki/Lahmu","Lahmu","Mitología mesopotámica","Gigante"],
           ["https://es.wikipedia.org/wiki/Mummu","Mummu","Mitología mesopotámica","Deidad"],
           ["https://es.wikipedia.org/wiki/Enkimdu","Enkimdu","Mitología sumeria","Dios"],
           ["https://es.wikipedia.org/wiki/Damkina","Damkina","Mitología mesopotámica","Diosa"],
           ["https://es.wikipedia.org/wiki/Emesh","Emesh","Mitología mesopotámica","Dios"],
           ["https://es.wikipedia.org/wiki/Enbilulu","Enbilulu","Mitología mesopotámica","Dios"],
           ["https://es.wikipedia.org/wiki/Enten","Enten","Mitología sumeria","Dios"],
           ["https://es.wikipedia.org/wiki/Erra","Erra","Mitología mesopotámica","Dios"],
           ["https://es.wikipedia.org/wiki/Geshtu-e","Geshtu-e","Mitología sumeria","Dios"],
           ["https://es.wikipedia.org/wiki/Gugalanna","Gugalanna","Mitología sumeria","Deidad"],
           ["https://es.wikipedia.org/wiki/Isimud","Isimud","Mitología mesopotámica","Dios"],
           ["https://es.wikipedia.org/wiki/Musdamma","Musdamma","Mitología mesopotámica","Dios"],
           ["https://es.wikipedia.org/wiki/Namtar","Namtar","Mitología mesopotámica","Dios"],
           ["https://es.wikipedia.org/wiki/Ninazu","Ninazu","Mitología sumeria","Dios"],
           ["https://es.wikipedia.org/wiki/Ninkasi","Ninkasi","Mitología mesopotámica","Diosa"],
           ["https://es.wikipedia.org/wiki/Ninkur","Ninkur","Mitología mesopotámica","Diosa"],
           ["https://es.wikipedia.org/wiki/Ninlil","Ninlil","Mitología sumeria","Diosa"],
           ["https://es.wikipedia.org/wiki/Ninsar","Ninsar","Mitología mesopotámica","Diosa"],
           ["https://es.wikipedia.org/wiki/Ninsikila","Ninsikila","Mitología mesopotámica","Diosa"],
           ["https://es.wikipedia.org/wiki/Ninsuna","Ninsuna","Mitología mesopotámica","Diosa"],
           ["https://es.wikipedia.org/wiki/Antum","Antum","Mitología sumeria","Diosa"],
           ["https://es.wikipedia.org/wiki/Nusku","Nusku","Mitología mesopotámica","Dios"],
           ["https://es.wikipedia.org/wiki/Tišpak","Tišpak","Mitología mesopotámica","Dios"],
           ["https://es.wikipedia.org/wiki/Uras","Uras","Mitología sumeria","Diosa"],
           ["https://es.wikipedia.org/wiki/Emperador_Ku","Emperador Ku","Mitología china","Personaje"],
           ["https://es.wikipedia.org/wiki/Shaohao","Shaohao","Mitología china","Personaje"],
           ["https://es.wikipedia.org/wiki/Zhuanxu","Zhuanxu","Mitología china","Personaje"],
           ["https://es.wikipedia.org/wiki/Emperador_Yao","Emperador Yao","Mitología china","Personaje"],
           ["https://es.wikipedia.org/wiki/Emperador_Shun","Emperador Shun","Mitología china","Personaje"],
           ["https://es.wikipedia.org/wiki/Yu_el_Grande","Yu el Grande","Mitología china","Personaje"],
           ["https://es.wikipedia.org/wiki/Fushoulu","Fushoulu","Mitología china","Dios"],
           ["https://es.wikipedia.org/wiki/Sun_Wukong","Sun Wukong","Mitología china","Héroe"],
           ["https://es.wikipedia.org/wiki/Bóreas","Bóreas","Mitología griega","Dios"],
           ["https://es.wikipedia.org/wiki/Céfiro","Céfiro","Mitología griega","Dios"],
           ["https://es.wikipedia.org/wiki/Cibeles","Cibeles","Mitología griega","Diosa"]]

personajes = url_pers.append(pd.DataFrame(new_pers, columns = ["URL", "Nombre", "Mitología", "Clase"])).reset_index(drop = True)

personajes

,URL,Nombre,Mitología,Clase
0,https://es.wikipedia.org/wiki/Aken_(dios),Aken,Mitología egipcia,Dios
1,https://es.wikipedia.org/wiki/Aker,Aker,Mitología egipcia,Dios
2,https://es.wikipedia.org/wiki/Amenhotep_(hijo_...,Amenhotep (hijo de Hapu),Mitología egipcia,Dios
3,https://es.wikipedia.org/wiki/Am%C3%B3n,Amón,Mitología egipcia,Dios
4,https://es.wikipedia.org/wiki/Amset,Amset,Mitología egipcia,Dios
...,...,...,...,...
4339,https://es.wikipedia.org/wiki/Fushoulu,Fushoulu,Mitología china,Dios
4340,https://es.wikipedia.org/wiki/Sun_Wukong,Sun Wukong,Mitología china,Héroe
4341,https://es.wikipedia.org/wiki/Bóreas,Bóreas,Mitología griega,Dios
4342,https://es.wikipedia.org/wiki/Céfiro,Céfiro,Mitología griega,Dios


### Unión de personajes con región

A continuación se quiere unir el dataset de los personajes con el de la región de cada mitología.

In [5]:
mitologias = pd.read_csv("../data/mitologias.csv")

mitologias

,Mitología,Región,Continente
0,Mitología lugbara,África Central,África
1,Mitología egipcia,África del Norte,África
2,Mitología china,Asia del Noreste,Asia
3,Mitología japonesa,Asia del Noreste,Asia
4,Mitología coreana,Asia del Noreste,Asia
...,...,...,...
210,Mitología yoruba,África del Oeste,África
211,Mitología zulú,África del Sur,África
212,Mitología zuñi,América del Norte,América
213,Mitología ática,Europa del Sur,Europa


Existen mismas mitologías con nombres diferentes, por ejemplo, "Mitología de América" y "Mitología americana". Así que primero estandarizamos los nombres.

In [6]:
std_dict = {"Mitología de América" : "Mitología americana",
            "Mitología de Asia" : "Mitología asiática",
            "Mitología de Europa" : "Mitología europea",
            "Mitología de Levante" : "Mitología levantina",
            "Mitología de África" : "Mitología africana",
            "Mitología del País Vasco" : "Mitología vasca",
            "Mitología del hinduismo" : "Mitología hinduista",
            "Mitología griega de Corinto" : "Mitología griega",
            "Mitología griega de Tesalia" : "Mitología griega",
            "Mitología griega de Tracia" : "Mitología griega"}

personajes = personajes.replace({"Mitología" : std_dict})
personajes

,URL,Nombre,Mitología,Clase
0,https://es.wikipedia.org/wiki/Aken_(dios),Aken,Mitología egipcia,Dios
1,https://es.wikipedia.org/wiki/Aker,Aker,Mitología egipcia,Dios
2,https://es.wikipedia.org/wiki/Amenhotep_(hijo_...,Amenhotep (hijo de Hapu),Mitología egipcia,Dios
3,https://es.wikipedia.org/wiki/Am%C3%B3n,Amón,Mitología egipcia,Dios
4,https://es.wikipedia.org/wiki/Amset,Amset,Mitología egipcia,Dios
...,...,...,...,...
4339,https://es.wikipedia.org/wiki/Fushoulu,Fushoulu,Mitología china,Dios
4340,https://es.wikipedia.org/wiki/Sun_Wukong,Sun Wukong,Mitología china,Héroe
4341,https://es.wikipedia.org/wiki/Bóreas,Bóreas,Mitología griega,Dios
4342,https://es.wikipedia.org/wiki/Céfiro,Céfiro,Mitología griega,Dios


Unimos los datasets.

In [7]:
personajes = personajes.merge(mitologias, on = "Mitología", how = "left")

personajes

,URL,Nombre,Mitología,Clase,Región,Continente
0,https://es.wikipedia.org/wiki/Aken_(dios),Aken,Mitología egipcia,Dios,África del Norte,África
1,https://es.wikipedia.org/wiki/Aker,Aker,Mitología egipcia,Dios,África del Norte,África
2,https://es.wikipedia.org/wiki/Amenhotep_(hijo_...,Amenhotep (hijo de Hapu),Mitología egipcia,Dios,África del Norte,África
3,https://es.wikipedia.org/wiki/Am%C3%B3n,Amón,Mitología egipcia,Dios,África del Norte,África
4,https://es.wikipedia.org/wiki/Amset,Amset,Mitología egipcia,Dios,África del Norte,África
...,...,...,...,...,...,...
4339,https://es.wikipedia.org/wiki/Fushoulu,Fushoulu,Mitología china,Dios,Asia del Noreste,Asia
4340,https://es.wikipedia.org/wiki/Sun_Wukong,Sun Wukong,Mitología china,Héroe,Asia del Noreste,Asia
4341,https://es.wikipedia.org/wiki/Bóreas,Bóreas,Mitología griega,Dios,Europa del Sur,Europa
4342,https://es.wikipedia.org/wiki/Céfiro,Céfiro,Mitología griega,Dios,Europa del Sur,Europa


### One-hot Encoding

Hay personajes repetidos, es decir, personajes que pertenecen a distintas Mitologías o a distintas Clases. Por lo tanto, cambiamos el formato de las variables "Mitología", "Clase", "Region" y "Continent" a One-Hot Encoding.

In [8]:
personajes.groupby(by = ["URL", "Nombre"]).size().sort_values(ascending = False)

URL                                                            Nombre                  
https://es.wikipedia.org/wiki/Humbaba                          Humbaba                     12
https://es.wikipedia.org/wiki/Enlil                            Enlil                       11
https://es.wikipedia.org/wiki/El_(dios_sem%C3%ADtico)          El (dios semítico)           8
https://es.wikipedia.org/wiki/Siguanaba                        Siguanaba                    8
https://es.wikipedia.org/wiki/Baal                             Baal                         8
                                                                                           ..
https://es.wikipedia.org/wiki/Maristino                        Maristino                    1
https://es.wikipedia.org/wiki/Maris_(mitolog%C3%ADa)           Maris                        1
https://es.wikipedia.org/wiki/Mari_(diosa)                     Mari (diosa)                 1
https://es.wikipedia.org/wiki/Marat%C3%B3n_(hijo_de_Epopeo)    Mar

In [9]:
ohe = (pd
       .get_dummies(personajes[["Mitología", "Clase", "Región", "Continente"]],
                    prefix = {"Mitología" : "", "Clase" : "Clase", "Región" : "Región", "Continente" : "Continente"},
                    prefix_sep = {"Mitología" : "", "Clase" : ":", "Región" : ":", "Continente" : ":"}))
personajes = (pd.
        concat([personajes, ohe], axis = 1).
        drop(columns = ["Mitología", "Clase", "Clase:Personaje", "Región", "Continente"]).
        groupby(by = ["URL", "Nombre"]).
        agg(max).
        reset_index())

personajes

,URL,Nombre,Mitología Báltica,Mitología aborigen australiana,Mitología abrahámica,Mitología acadia,Mitología africana,Mitología afroamericana,Mitología aimara,Mitología amazónica,...,Región:África Central,Región:África del Este,Región:África del Norte,Región:África del Oeste,Región:África del Sur,Continente:América,Continente:Asia,Continente:Europa,Continente:Oceanía,Continente:África
0,https://es.wikipedia.org/wiki/%C3%81ctor_(hijo...,Áctor (hijo de Deión),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,https://es.wikipedia.org/wiki/%C3%81ctor_(rey_...,Áctor (rey de Ftía),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,https://es.wikipedia.org/wiki/%C3%81diti,Áditi,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,https://es.wikipedia.org/wiki/%C3%81gave,Ágave,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,https://es.wikipedia.org/wiki/%C3%81gave_(mito...,Ágave,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3184,https://es.wikipedia.org/wiki/Zonget,Zonget,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3185,https://es.wikipedia.org/wiki/Zorra_teumesia,Zorra teumesia,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3186,https://es.wikipedia.org/wiki/Zorro_de_nueve_c...,Zorro de nueve colas,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3187,https://es.wikipedia.org/wiki/Zurvan,Zurvan,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


### Limpieza

En este apartado se hará una limpieza general del dataframe, esto es, modificación de variables y eliminación de observaciones.

* La clase "Deidad" agrupa las clases "Dios" y "Diosa", por lo que mutamos la variable "Clase:Deidad" para que se tenga esto en cuenta.

In [10]:
personajes.loc[(personajes["Clase:Dios"] == 1) | (personajes["Clase:Diosa"] == 1), "Clase:Deidad"] = 1

* Un artículo de Cloris y otro de Ágave son artículos de desambiguación. En el subapartado siguiente se eliminarán del dataset.
* Un artículo de Aletes pertenece a un personaje histórico, por lo que se elimina del dataset.
* Artículos que son anexos son eliminados del dataset.
* Artículos referentes a los personajes mitológicos de DC Cómics son eliminados del dataset.
* Artículos que hacen referencia a estatuas son eliminados del dataset.
* Ártículos que hacen referencia a eventos mitológicos son eliminados del dataset.

In [11]:
personajes.groupby("Nombre").size().sort_values(ascending = False)

Nombre
Cloris                        2
Ágave                         2
Aletes                        2
Šakkan                        1
Esténelo (hijo de Capaneo)    1
                             ..
Nibiru (planeta)              1
Nian                          1
Nhialac                       1
Ngenechén                     1
Aba                           1
Length: 3186, dtype: int64

In [12]:
no_pers = ["Anexo:", "DC_Comics", "Estatua",
           "Aletes_(deidad)", # este articulo pertenece a un personaje historico
           "Guerra_entre_los_Æsir_y_los_Vanir"] # este articulo pertenece a un evento

In [13]:
personajes = (personajes[[not(any(substring in url for substring in no_pers)) for url in personajes["URL"]]]
              .reset_index(drop = True))

personajes

,URL,Nombre,Mitología Báltica,Mitología aborigen australiana,Mitología abrahámica,Mitología acadia,Mitología africana,Mitología afroamericana,Mitología aimara,Mitología amazónica,...,Región:África Central,Región:África del Este,Región:África del Norte,Región:África del Oeste,Región:África del Sur,Continente:América,Continente:Asia,Continente:Europa,Continente:Oceanía,Continente:África
0,https://es.wikipedia.org/wiki/%C3%81ctor_(hijo...,Áctor (hijo de Deión),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,https://es.wikipedia.org/wiki/%C3%81ctor_(rey_...,Áctor (rey de Ftía),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,https://es.wikipedia.org/wiki/%C3%81diti,Áditi,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,https://es.wikipedia.org/wiki/%C3%81gave,Ágave,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,https://es.wikipedia.org/wiki/%C3%81gave_(mito...,Ágave,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3175,https://es.wikipedia.org/wiki/Zonget,Zonget,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3176,https://es.wikipedia.org/wiki/Zorra_teumesia,Zorra teumesia,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3177,https://es.wikipedia.org/wiki/Zorro_de_nueve_c...,Zorro de nueve colas,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3178,https://es.wikipedia.org/wiki/Zurvan,Zurvan,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


### Obtención del texto de los personajes

Ahora obtendremos el texto de todos los artículos de personajes mitológicos. Aprovechando que se leerán todos los artículos, también se obtienen los links a otros artículos del dataset, y se localizan las páginas pertenecientes a la categoría "desambiguación" ya que estos artículos no nos interesan.

In [14]:
# Funcion para limpiar el texto

def clean_text(text):
    text = re.sub("[^A-Za-zÀ-ÿ\u00f1\u00d1]", " ", text)
    text = text.lower()
    out = ""
    for word in text.split():
        out += " " + word
    return(out.strip())

In [15]:
# Funcion para obtener todo el texto util del articulo de wikipedia
# Se omiten los apartados de "Enlaces externos", "Referencias", "Fuentes", "Notas", "Vease tambien" y "Bibliografia"

omit = ["enlaces externos", "referencias", "fuentes", "notas", "véase también", "bibliografía"]

def obt_texto(url):
    pag = get_url(url)
    nodos = pag.find("div", class_ = "mw-parser-output").contents
    texto = ""
    h2 = True
    h3 = True
    for i in range(len(nodos)):
        if isinstance(nodos[i], str):
            continue
        if nodos[i].get("class") is not None:
            if ["noprint", "VT rellink", "ext-quick-survey-panel"] in nodos[i].get("class") or "VT ":
                continue
        tipo = nodos[i].name
        if tipo == "h2":
            if len(set([nodos[i].text.lower().find(o) for o in omit])) == 1: # si el string no contiene ninguna de las palabras clave...
                h2 = True
                h3 = True
            else: # si el string contiene alguna de las palabras que se quieren omitir...
                h2 = False
        if tipo == "h3":
            if len(set([nodos[i].text.find(o) for o in omit])) == 1:
                h3 = True
            else:
                h3 = False
        if h2 and h3:
            texto += nodos[i].text
    des = False
    if pag.find("div", class_ = "rellink noprint mw-disambig-page hatnote") is not None:
        des = True
    return({"texto" : clean_text(texto), "desambiguacion" : des})

In [16]:
# Funcion para obtener los links dirigidos a otros personajes

def row_matriz_ady(url):
    row = [0] * len(personajes)
    a_list = get_url(url).find("div", class_ = "mw-parser-output").find_all("a")
    links = ["https://es.wikipedia.org" + str(link.get("href")) for link in a_list]
    for link in links:
        if link in list(personajes["URL"]):
            row[personajes.index[personajes["URL"] == link][0]] = 1
    df = pd.DataFrame([row], index = [url], columns = personajes["Nombre"])
    return df

**Tarda mucho en ejecutarse**

In [17]:
textos = []
ady_matrix = pd.DataFrame()
no_desambiguacion = []
for url in personajes["URL"]:
    func_texto = obt_texto(url)
    textos.append(func_texto["texto"])
    no_desambiguacion.append(not(func_texto["desambiguacion"]))
    ady_matrix = ady_matrix.append(row_matriz_ady(url))

personajes["Texto"] = textos

In [18]:
personajes = personajes[no_desambiguacion]

personajes

,URL,Nombre,Mitología Báltica,Mitología aborigen australiana,Mitología abrahámica,Mitología acadia,Mitología africana,Mitología afroamericana,Mitología aimara,Mitología amazónica,...,Región:África del Este,Región:África del Norte,Región:África del Oeste,Región:África del Sur,Continente:América,Continente:Asia,Continente:Europa,Continente:Oceanía,Continente:África,Texto
0,https://es.wikipedia.org/wiki/%C3%81ctor_(hijo...,Áctor (hijo de Deión),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,en la mitología griega áctor era padre de mene...
1,https://es.wikipedia.org/wiki/%C3%81ctor_(rey_...,Áctor (rey de Ftía),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,en la mitología griega áctor en griego antiguo...
2,https://es.wikipedia.org/wiki/%C3%81diti,Áditi,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,en el marco del vedismo áditi es una diosa mad...
3,https://es.wikipedia.org/wiki/%C3%81gave,Ágave,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,en la mitología griega ágave en griego antiguo...
5,https://es.wikipedia.org/wiki/%C3%81lfr%C3%B6%...,Álfröðull,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,álfröðull alfrodhul del nórdico antiguo gloria...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3175,https://es.wikipedia.org/wiki/Zonget,Zonget,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,zonget es la diosa de la naturaleza en la mito...
3176,https://es.wikipedia.org/wiki/Zorra_teumesia,Zorra teumesia,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,en la mitología griega la zorra teumesia en gr...
3177,https://es.wikipedia.org/wiki/Zorro_de_nueve_c...,Zorro de nueve colas,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,el zorro de nueve colas o espíritu zorro es un...
3178,https://es.wikipedia.org/wiki/Zurvan,Zurvan,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,zurvan o zervan del avéstico zruvan tiempo en ...


In [19]:
ady_matrix = ady_matrix.iloc[no_desambiguacion, no_desambiguacion]

ady_matrix

Nombre,Áctor (hijo de Deión),Áctor (rey de Ftía),Áditi,Ágave,Álfröðull,Álope,Ámbrax,Ámico,Ántifo (hijo de Mirmidón),Ápate,...,Zhuanxu,Zibelthiurdos,Zipacná,Ziz,Zlatorog,Zonget,Zorra teumesia,Zorro de nueve colas,Zurvan,Ñamandú
https://es.wikipedia.org/wiki/%C3%81ctor_(hijo_de_Dei%C3%B3n),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://es.wikipedia.org/wiki/%C3%81ctor_(rey_de_Ft%C3%ADa),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://es.wikipedia.org/wiki/%C3%81diti,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://es.wikipedia.org/wiki/%C3%81gave,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://es.wikipedia.org/wiki/%C3%81lfr%C3%B6%C3%B0ull,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://es.wikipedia.org/wiki/Zonget,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://es.wikipedia.org/wiki/Zorra_teumesia,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://es.wikipedia.org/wiki/Zorro_de_nueve_colas,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://es.wikipedia.org/wiki/Zurvan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
personajes.to_csv("../data/personajes.csv", header = True, index = False)
ady_matrix.to_csv("../data/ady_matrix.csv", header = True, index = False)